In [6]:
import jax
import emlp
from emlp import EMLP, EMLPH, MLPH
from emlp.reps import Scalar, T
import sys
sys.path.append("../trainer/")
from hamiltonian_dynamics import DoubleSpringPendulum, BHamiltonianFlow, HamiltonianDataset, WindyDoubleSpringPendulum
import jax.numpy as jnp
sys.path.append("..")
from soft_emlp import MixedEMLP, MixedEMLPH
from emlp.groups import SO,O,S,Z,Trivial
from emlp.groups import SO2eR3,O2eR3,DkeR3

from torch.utils.data import DataLoader
from utils import LoaderTo
from oil.datasetup.datasets import split_dataset

In [2]:
%matplotlib notebook

In [3]:
# split={'train':10,'val':.1,'test':.1}

# base_ds = DoubleSpringPendulum(n_systems=100, chunk_len=50)
# datasets = split_dataset(base_ds,splits=split)
# dataloaders = {k:LoaderTo(DataLoader(v,batch_size=min(10,len(v)),shuffle=(k=='train'),
#                 num_workers=0,pin_memory=False)) for k,v in datasets.items()}

In [4]:
# base_ds.animate()

In [8]:
base_ds = WindyDoubleSpringPendulum(n_systems=100, chunk_len=50)
from IPython.display import HTML
HTML(WindyDoubleSpringPendulum().animate())

<IPython.core.display.Javascript object>

In [9]:

model = MixedEMLPH(base_ds.rep_in, Scalar, group=base_ds.symmetry)

In [11]:
model

MixedEMLPH

In [ ]:
(z0, ts), z = next(iter(dataloaders['train']))
BHamiltonianFlow(model, z0, ts[0])

In [ ]:
class DoubleSpringPendulum(HamiltonianDataset):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.rep_in = 4*T(1)#Vector
        self.rep_out = T(0)#Scalar
        self.symmetry = O2eR3()
        self.stats = (0,1,0,1)
    def H(self,z):
        g=1.5
        m1,m2,k1,k2,l1,l2 = 1,1,1,1,1,1
        x,p = unpack(z)
        p1,p2 = unpack(p)

        x1,x2 = unpack(x)
        ke = .5*(p1**2).sum(-1)/m1 + .5*(p2**2).sum(-1)/m2
        pe = .5*k1*(jnp.sqrt((x1**2).sum(-1))-l1)**2 
        pe += k2*(jnp.sqrt(((x1-x2)**2).sum(-1))-l2)**2
        pe += m1*g*x1[...,2]+m2*g*x2[...,2]
        return (ke + pe).sum()
    def sample_initial_conditions(self,bs):
        x1 = np.array([0,0,-1.5]) +.2*np.random.randn(bs,3)
        x2= np.array([0,0,-3.]) +.2*np.random.randn(bs,3)
        p = .4*np.random.randn(bs,6)
        z0 = np.concatenate([x1,x2,p],axis=-1)
        return z0
    @property
    def animator(self):
        return CoupledPendulumAnimation

In [ ]:
def unpack(z):
    D = jnp.shape(z)[-1]
    assert D % 2 == 0
    d = D//2
    q, p_or_v = z[..., :d], z[..., d:]
    return q, p_or_v

def hamiltonian_dynamics(hamiltonian, z,t):
    grad_h = grad(hamiltonian)
    gh = grad_h(z)
    return symplectic_form(gh)
def symplectic_form(z):
    q, p = unpack(z)
    return pack(p, -q)
def pack(q, p_or_v):
    return jnp.concatenate([q, p_or_v], axis=-1)

In [ ]:
ds = DoubleSpringPendulum()

In [ ]:
from jax import grad, jit, vmap, jacfwd, jvp, vjp
from functools import partial
from itertools import islice


In [ ]:
(z0, t), z = ds[0]

In [ ]:
dynamics = jit(vmap(jit(partial(hamiltonian_dynamics,ds.H)),(0,None)))

In [ ]:
dynamics(z0[None, :], t).shape

In [ ]:
hamiltonian_dynamics(ds.H, z0, t).shape

In [ ]:
grad(ds.H)(z0)

In [ ]:
t.shape

In [ ]:
z0.shape

In [ ]:
g=1
m1,m2,k1,k2,l1,l2 = 1,1,1,1,1,1

In [ ]:
x,p = unpack(z0)
p1,p2 = unpack(p)

In [ ]:
x.shape

In [ ]:
print("P1 = ", p1)
print("P2 = ", p2)

In [ ]:
x1,x2 = unpack(x)

In [ ]:
wind = jnp.array([1., 0.2, 0.])

In [ ]:
testx = x1[:, jnp.newaxis].repeat(10, -1).T

In [ ]:
testx @ wind

In [ ]:
x2.shape

In [ ]:
ke = .5*(p1**2).sum(-1)/m1 + .5*(p2**2).sum(-1)/m2
pe = .5*k1*(jnp.sqrt((x1**2).sum(-1))-l1)**2 
pe += k2*(jnp.sqrt(((x1-x2)**2).sum(-1))-l2)**2
pe += m1*g*x1[...,2]+m2*g*x2[...,2]

In [ ]:
wind.dot(x1)

In [ ]:
ke

In [ ]:
x1[..., 2]

In [ ]:
x1